In [9]:
## Parse PDB files into a dictionary
# use parse_multiple_chains.py; parse_multiple_chains.sh

In [10]:
import matplotlib.pyplot as plt

In [11]:
from dateutil import parser
import numpy as np
import os, time, gzip, json
import glob 
import argparse

# argparser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# argparser.add_argument("--pdb_folder", type=str, help="Path to a folder with pdb files, e.g. /home/my_pdbs/")
# argparser.add_argument("--out_path", type=str, help="Path where to save .jsonl dictionary of parsed pdbs")

# args = argparser.parse_args()

# folder_with_pdbs_path = args.pdb_folder
# save_path = args.out_path

#MODIFY THIS PATH TO YOUR FOLDER WITH PDBS
folder_with_pdbs_path = '/home/justas/projects/lab_github/mpnn/pdbs'
#MODIFY OUTPUT PATH
save_path = '/home/justas/projects/lab_github/mpnn/data/pdbs.jsonl'

alpha_1 = list("ARNDCQEGHILKMFPSTWYV-")
states = len(alpha_1)
alpha_3 = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE',
           'LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL','GAP']

aa_1_N = {a:n for n,a in enumerate(alpha_1)}
aa_3_N = {a:n for n,a in enumerate(alpha_3)}
aa_N_1 = {n:a for n,a in enumerate(alpha_1)}
aa_1_3 = {a:b for a,b in zip(alpha_1,alpha_3)}
aa_3_1 = {b:a for a,b in zip(alpha_1,alpha_3)}

def AA_to_N(x):
  # ["ARND"] -> [[0,1,2,3]]
  x = np.array(x);
  if x.ndim == 0: x = x[None]
  return [[aa_1_N.get(a, states-1) for a in y] for y in x]

def N_to_AA(x):
  # [[0,1,2,3]] -> ["ARND"]
  x = np.array(x);
  if x.ndim == 1: x = x[None]
  return ["".join([aa_N_1.get(a,"-") for a in y]) for y in x]


def parse_PDB_biounits(x, atoms=['N','CA','C'], chain=None):
  '''
  input:  x = PDB filename
          atoms = atoms to extract (optional)
  output: (length, atoms, coords=(x,y,z)), sequence
  '''
  xyz,seq,min_resn,max_resn = {},{},1e6,-1e6
  for line in open(x,"rb"):
    line = line.decode("utf-8","ignore").rstrip()

    if line[:6] == "HETATM" and line[17:17+3] == "MSE":
      line = line.replace("HETATM","ATOM  ")
      line = line.replace("MSE","MET")

    if line[:4] == "ATOM":
      ch = line[21:22]
      if ch == chain or chain is None:
        atom = line[12:12+4].strip()
        resi = line[17:17+3]
        resn = line[22:22+5].strip()
        x,y,z = [float(line[i:(i+8)]) for i in [30,38,46]]

        if resn[-1].isalpha(): 
            resa,resn = resn[-1],int(resn[:-1])-1
        else: 
            resa,resn = "",int(resn)-1
#         resn = int(resn)
        if resn < min_resn: 
            min_resn = resn
        if resn > max_resn: 
            max_resn = resn
        if resn not in xyz: 
            xyz[resn] = {}
        if resa not in xyz[resn]: 
            xyz[resn][resa] = {}
        if resn not in seq: 
            seq[resn] = {}
        if resa not in seq[resn]: 
            seq[resn][resa] = resi

        if atom not in xyz[resn][resa]:
          xyz[resn][resa][atom] = np.array([x,y,z])

  # convert to numpy arrays, fill in missing values
  seq_,xyz_ = [],[]
  try:
      for resn in range(min_resn,max_resn+1):
        if resn in seq:
          for k in sorted(seq[resn]): seq_.append(aa_3_N.get(seq[resn][k],20))
        else: seq_.append(20)
        if resn in xyz:
          for k in sorted(xyz[resn]):
            for atom in atoms:
              if atom in xyz[resn][k]: xyz_.append(xyz[resn][k][atom])
              else: xyz_.append(np.full(3,np.nan))
        else:
          for atom in atoms: xyz_.append(np.full(3,np.nan))
      return np.array(xyz_).reshape(-1,len(atoms),3), N_to_AA(np.array(seq_))
  except TypeError:
      return 'no_chain', 'no_chain'



pdb_dict_list = []
c = 0

if folder_with_pdbs_path[-1]!='/':
    folder_with_pdbs_path = folder_with_pdbs_path+'/'


init_alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I', 'J','K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V','W','X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j','k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't','u', 'v','w','x', 'y', 'z']
extra_alphabet = [str(item) for item in list(np.arange(300))]
chain_alphabet = init_alphabet + extra_alphabet

biounit_names = glob.glob(folder_with_pdbs_path+'*.pdb')
for biounit in biounit_names:
    my_dict = {}
    s = 0
    concat_seq = ''
    concat_N = []
    concat_CA = []
    concat_C = []
    concat_O = []
    concat_mask = []
    coords_dict = {}
    for letter in chain_alphabet:
        xyz, seq = parse_PDB_biounits(biounit, atoms=['N','CA','C','O'], chain=letter)
        if type(xyz) != str:
            concat_seq += seq[0]
            my_dict['seq_chain_'+letter]=seq[0]
            coords_dict_chain = {}
            coords_dict_chain['N_chain_'+letter]=xyz[:,0,:].tolist()
            coords_dict_chain['CA_chain_'+letter]=xyz[:,1,:].tolist()
            coords_dict_chain['C_chain_'+letter]=xyz[:,2,:].tolist()
            coords_dict_chain['O_chain_'+letter]=xyz[:,3,:].tolist()
            my_dict['coords_chain_'+letter]=coords_dict_chain
            s += 1
    fi = biounit.rfind("/")
    my_dict['name']=biounit[(fi+1):-4]
    my_dict['num_of_chains'] = s
    my_dict['seq'] = concat_seq
    if s < len(chain_alphabet):
        pdb_dict_list.append(my_dict)
        c+=1
        
        
with open(save_path, 'w') as f:
    for entry in pdb_dict_list:
        f.write(json.dumps(entry) + '\n')
        
print(f'Successfully finished: {len(biounit_names)} pdbs')


Successfully finished: 2 pdbs


In [12]:
#MAKE masked/visible chains dictionary
#Masked means the chains that need to be designed, e.g. binder
#Visible means the chains that are fixed, they won't be modelled but only used as a context, e.g. target
# use make_masked_visible_chain_dict.py; make_masked_visible_chain_dict.sh

In [13]:
import glob
import random

import json

#MODIFY THIS PATH - it is a path to the parsed pdb files
with open('/home/justas/projects/lab_github/mpnn/data/pdbs.jsonl', 'r') as json_file:
    json_list = list(json_file)

my_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = [item[-1:] for item in list(result) if item[:9]=='seq_chain'] #['A','B', 'C',...]
    masked_chain_list = ['A'] #predict sequence of chain A
    visible_chain_list = ['B'] #allow to use chain B as a context
    my_dict[result['name']]= (masked_chain_list, visible_chain_list)

#MODIFY THIS PATH FOR THE OUTPUT DICTIONARY
with open('/home/justas/projects/lab_github/mpnn/data/pdbs_masked.jsonl', 'w') as f:
    f.write(json.dumps(my_dict) + '\n')


print('Finished')
# Output looks like this:
# {"5TTA": [["A"], ["B"]], "3LIS": [["A"], ["B"]]}

Finished


In [14]:
#FIX some positions in the designed sequence, e.g. fixing interface residues

In [15]:
import glob
import random
import numpy as np
import json
import itertools

In [16]:
#itertools example to make a list
list(itertools.chain(list(np.arange(1,4)), list(np.arange(7,10)), [22, 25, 33]))

[1, 2, 3, 7, 8, 9, 22, 25, 33]

In [17]:
#FIX some positions
#use make_fixed_position_dict.py

In [18]:
import glob
import random
import numpy as np
import json
import itertools

#MODIFY this path
with open('/home/justas/projects/lab_github/mpnn/data/pdbs.jsonl', 'r') as json_file:
    json_list = list(json_file)

my_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = [item[-1:] for item in list(result) if item[:9]=='seq_chain']
    fixed_position_dict = {}
    print(result['name'])
    #FIX ONLY PDB NAMED 5TTA in the chain A
    if result['name'] == '5TTA':
        for chain in all_chain_list:
            if chain == 'A':
                fixed_position_dict[chain] = [int(item) for item in list(itertools.chain(list(np.arange(1,4)), list(np.arange(7,10)), [22, 25, 33]))]
            else:
                fixed_position_dict[chain] = []
    else:
        for chain in all_chain_list:
            fixed_position_dict[chain] = []
    my_dict[result['name']] = fixed_position_dict

#MODIFY this path   
with open('/home/justas/projects/lab_github/mpnn/data/pdbs_fixed.jsonl', 'w') as f:
    f.write(json.dumps(my_dict) + '\n')


print('Finished')
#e.g. output
#{"5TTA": {"A": [1, 2, 3, 7, 8, 9, 22, 25, 33], "B": []}, "3LIS": {"A": [], "B": []}}

5TTA
3LIS
Finished


In [19]:
#Tie positions together
#In the example below homo-dimer symmetry is created, {"A": [1], "B": [1]}, {"A": [2], "B": [2]}, {"A": [3], "B": [3]}...
# use make_tied_positions_dict.py

In [20]:
import glob
import random
import numpy as np
import json
import itertools

#MODIFY this path
with open('/home/justas/projects/lab_github/mpnn/data/pdbs.jsonl', 'r') as json_file:
    json_list = list(json_file)

my_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = sorted([item[-1:] for item in list(result) if item[:9]=='seq_chain']) #A, B, C, ...
    tied_positions_list = []
    if result['name'] == '3LIS':
        chain_length = len(result["seq_chain_A"])
        for i in range(1,chain_length+1):
            temp_dict = {}
            temp_dict[all_chain_list[0]] = [i]  #all_chain_list[0] == "A" in this case
            temp_dict[all_chain_list[1]] = [i]  #all_chain_list[0] == "B"
            tied_positions_list.append(temp_dict)
    else:
        tied_positions_list = []
    my_dict[result['name']] = tied_positions_list

#Write output to:    
with open('/home/justas/projects/lab_github/mpnn/data/pdbs_tied.jsonl', 'w') as f:
    f.write(json.dumps(my_dict) + '\n')


print('Finished')

#e.g. output
#{"5TTA": [], "3LIS": [{"A": [1], "B": [1]}, {"A": [2], "B": [2]}, {"A": [3], "B": [3]}, {"A": [4], "B": [4]}, {"A": [5], "B": [5]}, {"A": [6], "B": [6]}, {"A": [7], "B": [7]}, {"A": [8], "B": [8]}, {"A": [9], "B": [9]}, {"A": [10], "B": [10]}, {"A": [11], "B": [11]}, {"A": [12], "B": [12]}, {"A": [13], "B": [13]}, {"A": [14], "B": [14]}, {"A": [15], "B": [15]}, {"A": [16], "B": [16]}, {"A": [17], "B": [17]}, {"A": [18], "B": [18]}, {"A": [19], "B": [19]}, {"A": [20], "B": [20]}, {"A": [21], "B": [21]}, {"A": [22], "B": [22]}, {"A": [23], "B": [23]}, {"A": [24], "B": [24]}, {"A": [25], "B": [25]}, {"A": [26], "B": [26]}, {"A": [27], "B": [27]}, {"A": [28], "B": [28]}, {"A": [29], "B": [29]}, {"A": [30], "B": [30]}, {"A": [31], "B": [31]}, {"A": [32], "B": [32]}, {"A": [33], "B": [33]}, {"A": [34], "B": [34]}, {"A": [35], "B": [35]}, {"A": [36], "B": [36]}, {"A": [37], "B": [37]}, {"A": [38], "B": [38]}, {"A": [39], "B": [39]}, {"A": [40], "B": [40]}, {"A": [41], "B": [41]}, {"A": [42], "B": [42]}, {"A": [43], "B": [43]}, {"A": [44], "B": [44]}, {"A": [45], "B": [45]}, {"A": [46], "B": [46]}, {"A": [47], "B": [47]}, {"A": [48], "B": [48]}, {"A": [49], "B": [49]}, {"A": [50], "B": [50]}, {"A": [51], "B": [51]}, {"A": [52], "B": [52]}, {"A": [53], "B": [53]}, {"A": [54], "B": [54]}, {"A": [55], "B": [55]}, {"A": [56], "B": [56]}, {"A": [57], "B": [57]}, {"A": [58], "B": [58]}, {"A": [59], "B": [59]}, {"A": [60], "B": [60]}, {"A": [61], "B": [61]}, {"A": [62], "B": [62]}, {"A": [63], "B": [63]}, {"A": [64], "B": [64]}, {"A": [65], "B": [65]}, {"A": [66], "B": [66]}, {"A": [67], "B": [67]}, {"A": [68], "B": [68]}, {"A": [69], "B": [69]}, {"A": [70], "B": [70]}, {"A": [71], "B": [71]}, {"A": [72], "B": [72]}, {"A": [73], "B": [73]}, {"A": [74], "B": [74]}, {"A": [75], "B": [75]}, {"A": [76], "B": [76]}, {"A": [77], "B": [77]}, {"A": [78], "B": [78]}, {"A": [79], "B": [79]}, {"A": [80], "B": [80]}, {"A": [81], "B": [81]}, {"A": [82], "B": [82]}, {"A": [83], "B": [83]}, {"A": [84], "B": [84]}, {"A": [85], "B": [85]}, {"A": [86], "B": [86]}, {"A": [87], "B": [87]}, {"A": [88], "B": [88]}, {"A": [89], "B": [89]}, {"A": [90], "B": [90]}, {"A": [91], "B": [91]}, {"A": [92], "B": [92]}, {"A": [93], "B": [93]}, {"A": [94], "B": [94]}, {"A": [95], "B": [95]}, {"A": [96], "B": [96]}]}

Finished


In [21]:
#MAKE global bias dictiobary
# use make_bias_AA.py

In [22]:
import numpy as np
import json

my_dict = {"A": -0.01, "G": 0.02} #0.1 is a good value to start with

with open('/home/justas/projects/lab_github/mpnn/data/bias_AA.jsonl', 'w') as f:
    f.write(json.dumps(my_dict) + '\n')
    
#e.g. output
#{"A": -0.01, "G": 0.02}

In [23]:
#Omit AAs per position
# make_omit_AA.py

In [24]:
import glob
import random
import numpy as np
import json
import itertools

#MODIFY this path
with open('/home/justas/projects/lab_github/mpnn/data/pdbs.jsonl', 'r') as json_file:
    json_list = list(json_file)

my_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = [item[-1:] for item in list(result) if item[:9]=='seq_chain']
    fixed_position_dict = {}
    print(result['name'])
    if result['name'] == '5TTA':
        for chain in all_chain_list:
            if chain == 'A':
                fixed_position_dict[chain] = [
                    [[int(item) for item in list(itertools.chain(list(np.arange(1,4)), list(np.arange(7,10)), [22, 25, 33]))], 'GPL'],
                    [[int(item) for item in list(itertools.chain([40, 41, 42, 43]))], 'WC'],
                    [[int(item) for item in list(itertools.chain(list(np.arange(50,150))))], 'ACEFGHIKLMNRSTVWYX'],
                    [[int(item) for item in list(itertools.chain(list(np.arange(160,200))))], 'FGHIKLPQDMNRSTVWYX']]
            else:
                fixed_position_dict[chain] = []
    else:
        for chain in all_chain_list:
            fixed_position_dict[chain] = []
    my_dict[result['name']] = fixed_position_dict

#MODIFY this path   
with open('/home/justas/projects/lab_github/mpnn/data/omit_AA.jsonl', 'w') as f:
    f.write(json.dumps(my_dict) + '\n')


print('Finished')
#e.g. output
#{"5TTA": {"A": [[[1, 2, 3, 7, 8, 9, 22, 25, 33], "GPL"], [[40, 41, 42, 43], "WC"], [[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149], "ACEFGHIKLMNRSTVWYX"], [[160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], "FGHIKLPQDMNRSTVWYX"]], "B": []}, "3LIS": {"A": [], "B": []}}  

5TTA
3LIS
Finished


In [ ]:
#Make PSSM dictionary to bias MPNN
# use make_pssm_dict.py

In [30]:
import pandas as pd
import numpy as np

import glob
import random
import numpy as np
import json


def softmax(x, T):
    return np.exp(x/T)/np.sum(np.exp(x/T), -1, keepdims=True)

def parse_pssm(path):
    data = pd.read_csv(path, skiprows=2)
    floats_list_list = []
    for i in range(data.values.shape[0]):
        str1 = data.values[i][0][4:]
        floats_list = []
        for item in str1.split():
            floats_list.append(float(item))
        floats_list_list.append(floats_list)
    np_lines = np.array(floats_list_list)
    return np_lines

np_lines = parse_pssm('/home/swang523/RLcage/capsid/monomersfordesign/8-16-21/pssm_rainity_final_8-16-21_int/build_0.2089_0.98_0.4653_19_2.00_0.005745.pssm')

mpnn_alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
input_alphabet = 'ARNDCQEGHILKMFPSTWYV'

permutation_matrix = np.zeros([20,21])
for i in range(20):
    letter1 = input_alphabet[i]
    for j in range(21):
        letter2 = mpnn_alphabet[j]
        if letter1 == letter2:
            permutation_matrix[i,j]=1.

pssm_log_odds = np_lines[:,:20] @ permutation_matrix
pssm_probs = np_lines[:,20:40] @ permutation_matrix

X_mask = np.concatenate([np.zeros([1,20]), np.ones([1,1])], -1)

def softmax(x, T):
    return np.exp(x/T)/np.sum(np.exp(x/T), -1, keepdims=True)

#Load parsed PDBs:  
with open('/home/justas/projects/cages/parsed/test.jsonl', 'r') as json_file:
    json_list = list(json_file)

my_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = [item[-1:] for item in list(result) if item[:9]=='seq_chain']
    pssm_dict = {}
    for chain in all_chain_list:
        pssm_dict[chain] = {}
        pssm_dict[chain]['pssm_coef'] = (np.ones(len(result['seq_chain_A']))).tolist() #a number between 0.0 and 1.0 specifying how much attention put to PSSM, can be adjusted later as a flag
        pssm_dict[chain]['pssm_bias'] = (softmax(pssm_log_odds-X_mask*1e8, 1.0)).tolist() #PSSM like, [length, 21] such that sum over the last dimension adds up to 1.0
        pssm_dict[chain]['pssm_log_odds'] = (pssm_log_odds).tolist()
    my_dict[result['name']] = pssm_dict

#Write output to:    
with open('/home/justas/projects/lab_github/mpnn/data/pssm_dict.jsonl', 'w') as f:
    f.write(json.dumps(my_dict) + '\n')
